In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from doco.agents.supervisor import Supervisor
from doco.agents.summarizer.basic import Summarizer
from doco.agents.content_creator.basic import ContentCreator


supervisor_agent = Supervisor().build()
summarizer_agent = Summarizer().build()
contentcreator_agent = ContentCreator().build()

/home/octoopt/anaconda3/envs/llmapi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from doco.agents.utils import agent_node
from doco.io._graph import AgentState, members

import functools

summarizer_node = functools.partial(agent_node, agent=summarizer_agent, name="Summarizer")
contentcreator_node = functools.partial(agent_node, agent=contentcreator_agent, name='ContentCreator')

In [4]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(AgentState)

graph_builder.add_node("ContentCreator", contentcreator_node)
graph_builder.add_node("Summarizer", summarizer_node)
graph_builder.add_node("supervisor", supervisor_agent)


# Add entry point
graph_builder.add_edge(START, "supervisor")

for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    graph_builder.add_edge(member, "supervisor")


# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
graph_builder.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

graph = graph_builder.compile()

ValueError: Found edge starting at unknown node 'Summarizer'